#### Importing main libraries

In [5]:
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime,timedelta 


#### Defining functions

In [6]:
def make_dataset(data,df_info):
    df=data.dropna()
    df_info=df_info.dropna()
    df["genre_name"] = np.nan
    df["area_name"] = np.nan
    df["latitude"] = np.nan
    df["longitude"] = np.nan
    df["holiday"] = np.nan



    df.visit_datetime=df.visit_datetime.apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
    df.reserve_datetime=df.reserve_datetime.apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S') )

    df["time_difference"] =  (df.visit_datetime - df.reserve_datetime)/timedelta(hours=1)


    for i in range(np.shape(df_info)[0]):
        if np.isin(df_info.iloc[i,0],df.iloc[:,0]):
            store_loc=np.where(df.iloc[:,0] == str(df_info.iloc[i,0]))[0]
            df.loc[store_loc,["genre_name"]]=df_info.iloc[i,1]
            df.loc[store_loc,["area_name"]]=df_info.iloc[i,2]
            df.loc[store_loc,["latitude"]]=df_info.iloc[i,3]
            df.loc[store_loc,["longitude"]]=df_info.iloc[i,4]


    df["visit_dayofweek"] =  df.visit_datetime.apply(lambda x: datetime.weekday(x))
#    df["visit_year"] =  df.visit_datetime.apply(lambda x: x.year)
    df["visit_month"] =  df.visit_datetime.apply(lambda x: x.month)
    df["visit_day"] =  df.visit_datetime.apply(lambda x: x.day)
    df["visit_hour"] =  df.visit_datetime.apply(lambda x: x.hour)
#    df["reserve_dayofweek"] =  df.reserve_datetime.apply(lambda x: datetime.weekday(x))
#    df["reserve_year"] =  df.reserve_datetime.apply(lambda x: x.year)
#    df["reserve_month"] =  df.reserve_datetime.apply(lambda x: x.month)
#    df["reserve_day"] =  df.reserve_datetime.apply(lambda x: x.day)
#    df["reserve_hour"] =  df.reserve_datetime.apply(lambda x: x.hour)
    df["visit_date"]=df.visit_datetime.apply(lambda x: x.date())

    for i,item  in enumerate(holiday_data.calendar_date):
        if np.isin(df.visit_date,item).any():
            df.holiday[np.where(df.visit_date==item)[0]]=holiday_data.loc[i,'holiday_flg']
    df=df.dropna()
    return df



#### Path directory 

In [16]:
input_data_path='/Users/leilasapple/Documents/Springboard/recruit-restaurant-visitor-forecasting/input_files/'
output_data_path='/Users/leilasapple/Documents/Springboard/recruit-restaurant-visitor-forecasting/output_files/'

#### Importing data
Data have come from two major websites which provide air and hpg data sets. 

##### To read air data

In [9]:
air_reserve=pd.read_csv(os.path.join(input_data_path,'air_reserve.csv'))
air_store_info=pd.read_csv(os.path.join(input_data_path,'air_store_info.csv'))
air_visit_data=pd.read_csv(os.path.join(input_data_path,'air_visit_data.csv'))


##### To read hpg data

In [10]:
hpg_reserve=pd.read_csv(os.path.join(input_data_path,'hpg_reserve.csv'))
hpg_store_info=pd.read_csv(os.path.join(input_data_path,'hpg_store_info.csv'))


##### To read other data

In [11]:
store_id_relation=pd.read_csv(os.path.join(input_data_path,'store_id_relation.csv'))
date_info=pd.read_csv(os.path.join(input_data_path,'date_info.csv'))
submission_data=pd.read_csv(os.path.join(input_data_path,'sample_submission.csv'))
holiday_data=pd.read_csv(os.path.join(input_data_path,'date_info.csv'))


In [12]:
holiday_data.calendar_date=holiday_data.calendar_date.apply(lambda x:pd.to_datetime(x).date())


In [13]:
df_air=make_dataset(air_reserve,air_store_info)
df_air=df_air.rename(columns={"air_store_id": "store_id"})


/Users/leilasapple/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
hpg_reserve_=hpg_reserve.dropna()
hpg_store_info=hpg_store_info.dropna()
df_hpg=make_dataset(hpg_reserve_,hpg_store_info)
df_hpg=df_hpg.rename(columns={"hpg_store_id": "store_id"})
df_hpg=df_hpg.dropna()

/Users/leilasapple/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
for i, item in enumerate(store_id_relation.iloc[:,1]):
    if np.isin(df_hpg.iloc[:,0],item).any():
        hpg_loc=np.where(df_hpg.iloc[:,0]==item)[0]
        df_hpg.iloc[hpg_loc,0]=store_id_relation.iloc[i,0]
df=pd.concat([df_air,df_hpg],ignore_index=True)
print(df.describe())
print(df.head())

To save all data in a csv file.

In [ ]:
df.to_csv(os.path.join(output_data_path,'df0.csv'), encoding='utf-8',index=False)


To have id for every resturant in every day
We need to know how many customer went to each resturant


In [ ]:
df['store_id_date']=df.store_id.values+'_'+df.visit_date.values
uniq_store_date=df['store_id_date'].unique().astype(str)

Generate new set of data for the model
This data set has unique values for each resturant in every day

In [17]:
data=pd.read_csv(os.path.join(output_data_path,'data.csv'))


Grouping data by resturant and day 
data were grouped by the resurant and the day of visit. So every resturant-day is a group. Data were grouped by resurant id store and the day

In [18]:
grouped=df.groupby('store_id_date')

NameError: name 'df' is not defined

- Gener, area, holiday, day and month in every group are the same based on the grouping 
- Visitors in every group are summation of all visitors in each reservation for that group
- The time differnce for each group is taken as the average of time difference of all reservations in a that group 


In [ ]:
data['store_id']=data.id.apply(lambda x: x[:-11])
data['genre']=data.id.apply(lambda x: grouped.get_group(x).iloc[0,4])
data['visitor']=data.id.apply(lambda x: grouped.get_group(x).iloc[:,3].sum())
data['area']=data.id.apply(lambda x: grouped.get_group(x).iloc[0,5])
data['holiday']=data.id.apply(lambda x: grouped.get_group(x).iloc[0,8])
data['dt']=data.id.apply(lambda x: grouped.get_group(x).iloc[:,9].mean())
data['day']=data.id.apply(lambda x: grouped.get_group(x).iloc[0,10])
data['month']=data.id.apply(lambda x: grouped.get_group(x).iloc[0,12])

In [ ]:
# backup the data
data.to_csv(os.path.join(output_data_path,'data.csv'), encoding='utf-8',index=False)
data=pd.read_csv(os.path.join(output_data_path,'data.csv'))


In [20]:
data.describe()

,visitor,holiday,dt,day,month
count,587957.000000,587957.000000,587957.000000,587957.000000,587957.000000
mean,8.610766,0.064949,167.435631,3.361385,5.728307
std,10.973159,0.246435,190.366523,1.836051,3.728686
min,1.000000,0.000000,0.000000,0.000000,1.000000
25%,3.000000,0.000000,46.000000,2.000000,3.000000
50%,5.000000,0.000000,107.000000,4.000000,4.000000
75%,10.000000,0.000000,219.500000,5.000000,9.000000
max,2241.000000,1.000000,8429.284314,6.000000,12.000000


In [19]:
##Defining a function for grouping the data and returning the value of the customers in the grouping 
def cutomer_features(feature,key_name,d1,d2):
    d2=d1
    g=d2.groupby(feature)
    d2[key_name]=np.nan
    for i ,item in enumerate(g.groups):
        a=True
        for j , ftr in feature:
            a=a & d2.loc[ftr]==item[j]
        LOC=np.where(a)[0]
        d2.loc[key_name]=g.get_group(item).loc[:'visitor'].mean()
    return d2

In [ ]:
#grouping data by the resturant only
grouped=data.groupby('store_id')
data['store_index']=np.nan
for i,item in enumerate(grouped.groups):
    loc_item=np.where(data['store_id']==item)[0]
    data.loc[loc_item,'store_index']=grouped.get_group(item).loc[:,'visitor'].mean()

In [ ]:
##### To check the effect of location on reservations  
grouped=data.groupby('area')
data['area_index']=np.nan
for i,item in enumerate(grouped.groups):
    loc_item=np.where(data['area']==item)[0]
    data.loc[loc_item,'area_index']=grouped.get_group(item).loc[:,'visitor'].mean()

In [ ]:
grouped=data.groupby('genre')
data['genre_index']=np.nan
for i,item in enumerate(grouped.groups):
    loc_item=np.where(data['genre']==item)[0]
    data.loc[loc_item,'genre_index']=grouped.get_group(item).loc[:,'visitor'].mean()

In [ ]:
##### To check the effect of genre and location on reservations   
grouped=data.groupby(['genre','area'])
data['genre_area_index']=np.nan
for i,item in enumerate(grouped.groups):
    a=data.genre==item[0]
    b=data.area==item[1]
    LOC=np.where(a & b)[0]
    data.loc[LOC,'genre_area_index']=grouped.get_group(item).loc[:,'visitor'].mean()

In [ ]:
##### To check the effect of the holiday on reservation 

grouped=data.groupby('holiday')
data['holiday_index']=np.nan
for i,item in enumerate(grouped.groups):
    loc_item=np.where(data['holiday']==item)[0]
    data.loc[loc_item,'holiday_index']=grouped.get_group(item).loc[:,'visitor'].mean()

In [ ]:
##### To check the effect of the holiday for each resturant on reservation 
grouped=data.groupby(['store_id','holiday'])
data['store_holiday_index']=np.nan
for i,item in enumerate(grouped.groups):
    a=data.store_id==item[0]
    b=data.holiday==item[1]
    LOC=np.where(a & b)[0]
    data.loc[LOC,'store_holiday_index']=grouped.get_group(item).loc[:,'visitor'].mean()

In [ ]:
##### To check the effect of the month on reservation 

grouped=data.groupby('month')
data['month_index']=np.nan
for i,item in enumerate(grouped.groups):
    loc_item=np.where(data['month']==item)[0]
    data.loc[loc_item,'month_index']=grouped.get_group(item).loc[:,'visitor'].mean()

In [ ]:
##### To check the effect of the month for each resturant on reservation 
grouped=data.groupby(['store_id','month'])
data['store_month_index']=np.nan
for i,item in enumerate(grouped.groups):
    a=data.store_id==item[0]
    b=data.month==item[1]
    LOC=np.where(a & b)[0]
    data.loc[LOC,'store_month_index']=grouped.get_group(item).loc[:,'visitor'].mean()

In [ ]:
##### To check the effect of the day of the week on reservation 

grouped=data.groupby('day')
data['day_index']=np.nan
for i,item in enumerate(grouped.groups):
    loc_item=np.where(data['day']==item)[0]
    data.loc[loc_item,'day_index']=grouped.get_group(item).loc[:,'visitor'].mean()

In [ ]:
##### To check the effect of the day of the week for each resturant on reservation 
grouped=data.groupby(['store_id','day'])
data['store_day_index']=np.nan
for i,item in enumerate(grouped.groups):
    a=data.store_id==item[0]
    b=data.day==item[1]
    LOC=np.where(a & b)[0]
    if i%5000==0:print(i,item,datetime.now())
    data.loc[LOC,'store_day_index']=grouped.get_group(item).loc[:,'visitor'].mean()

In [ ]:
##### To check the effect of the day of the week in each month on reservation 
grouped=data.groupby(['day','month'])
data['day_month_index']=np.nan
for i,item in enumerate(grouped.groups):
    a=data.day==item[0]
    b=data.month==item[1]
    LOC=np.where(a & b)[0]
    data.loc[LOC,'day_month_index']=grouped.get_group(item).loc[:,'visitor'].mean()

In [ ]:
##### To check the effect of the day og the week and the month for each resturant on reservation 
grouped=data.groupby(['store_id','day','month'])
data['store_day_month_index']=np.nan
for i,item in enumerate(grouped.groups):
    a=data.store_id==item[0]
    b=data.day==item[1]
    c=data.month==item[2]
    LOC=np.where(a & b & c)[0]
    if i%100 ==0: print(i,item)
    data.loc[LOC,'store_day_month_index']=grouped.get_group(item).loc[:,'visitor'].mean()

In [ ]:
grouped=data.groupby(['holiday','day','month'])
data['holiday_day_month_index']=np.nan
for i,item in enumerate(grouped.groups):
    
    a=data.holiday==item[0]
    b=data.day==item[1]
    c=data.month==item[2]
    LOC=np.where(a & b & c)[0]
    data.loc[LOC,'holiday_day_month_index']=grouped.get_group(item).loc[:,'visitor'].mean()

In [ ]:
print(data.describe())
print(data.head())

data.to_csv(os.path.join(output_data_path,'data1.csv'), encoding='utf-8',index=False)
data=pd.read_csv(os.path.join(output_data_path,'data1.csv'))